In [1]:
# Importing the required libaries

import os
from typing import List
from datetime import datetime
from google.colab import drive

import sqlite3
import pandas as pd
from tqdm import tqdm
import yfinance as yf

In [2]:
# Configuring libraries

tqdm.pandas()

In [3]:
# Mounting Google Drive to Colab runtime

drive.mount("/content/MyDrive")

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [4]:
print(f"Current Working Directory: {os.getcwd()}")

print("Changing the directory to the project root directory...")
os.chdir("/content/MyDrive/MyDrive/FAI - Project")

print(f"Current Working Directory: {os.getcwd()}")

Current Working Directory: /content
Changing the directory to the project root directory...
Current Working Directory: /content/MyDrive/MyDrive/FAI - Project


In [ ]:
from typing import List
from datetime import datetime
import sqlite3
import pandas as pd
import yfinance as yf
from tqdm import tqdm

class DataCollator:
    """
    A class for collecting and merging historical stock data and news headlines for a list of tickers.
    """

    def __init__(self, tickers: List[str], start_date: str,
                 end_date: str, news_db_url: str):
        """
        Constructor to initialize DataCollator object.

        Args:
            tickers (List[str]): List of stock tickers.
            start_date (str): Start date for historical data retrieval.
            end_date (str): End date for historical data retrieval.
            news_db_url (str): URL of the SQLite database for news data.
        """
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.news_db_url = news_db_url

    def get_historical_data(self):
        """
        Retrieves historical stock data for the specified tickers.

        Returns:
            pd.DataFrame: DataFrame containing historical stock data.
        """
        historical_df = pd.DataFrame()

        for ticker in self.tickers:
            ticker_df = yf.Ticker(ticker).history(
                start=self.start_date, end=self.end_date
            )
            ticker_df["Ticker"] = ticker
            ticker_df.index = pd.to_datetime(ticker_df.index.strftime("%Y-%m-%d"))

            historical_df = pd.concat(objs=[historical_df, ticker_df], axis=0)

        return historical_df

    def get_news_headlines(self, ticker: str):
        """
        Retrieves news headlines for a specific stock ticker.

        Args:
            ticker (str): Stock ticker.

        Returns:
            pd.DataFrame: DataFrame containing news headlines.
        """
        connection = sqlite3.connect(self.news_db_url)

        ticker_news_df = pd.read_sql_query(
            "SELECT * from {ticker}_".format(ticker=ticker), connection
        )

        # Additional processing for date and filtering
        ticker_news_df["date"] = ticker_news_df["datetime"].progress_apply(
            self.convert_unix_timestamp_to_datetime
        )
        ticker_news_df["date"] = pd.to_datetime(ticker_news_df["date"])
        ticker_news_df.sort_values(by="date")

        ticker_news_df = ticker_news_df[
            (ticker_news_df["date"] >= self.start_date) &
            (ticker_news_df["date"] <= self.end_date)
        ]

        # Drop unnecessary columns and group headlines by date
        ticker_news_df.drop(
            columns=["category", "datetime", "id",
                     "image", "related", "source", "url"],
            inplace=True
        )
        ticker_news_df = ticker_news_df.groupby(by="date")["headline"].apply(" | ".join).reset_index()
        ticker_news_df.columns = ["Date", "Headlines"]

        return ticker_news_df

    @staticmethod
    def convert_unix_timestamp_to_datetime(time_stamp: str) -> datetime:
        """
        Converts Unix timestamp to datetime format.

        Args:
            time_stamp (str): Unix timestamp.

        Returns:
            datetime: Datetime object.
        """
        return datetime.utcfromtimestamp(time_stamp).strftime("%Y-%m-%d")

    def merge_hist_and_news(self) -> pd.DataFrame:
        """
        Merges historical stock data and news headlines.

        Returns:
            pd.DataFrame: Merged DataFrame.
        """
        
        historical_df = self.get_historical_data()

        result_df = pd.DataFrame()
        for ticker in self.tickers:
            ticker_df = historical_df[historical_df.Ticker == ticker]

            # Call to get_news_headlines method
            ticker_news = collator.get_news_headlines(ticker=ticker)

            # Merge historical data with news headlines
            ticker_df = ticker_df.merge(right=ticker_news, how="left", on="Date")

            result_df = pd.concat([result_df, ticker_df], axis=0)

        return result_df

    def get_tickers_data(self):
        """
        Retrieves and returns merged data for all specified tickers.

        Returns:
            pd.DataFrame: Merged DataFrame for all tickers.
        """
        
        return self.merge_hist_and_news()

In [14]:
start_date = "2022-11-26"
end_date = "2023-11-15"

DATA_DIR = "data"
NEWS_DATA_DB = os.path.join(DATA_DIR, "financial_data.db")

tickers = ["AAPL", "ABNB", "AMZN", "COIN", "DUOL", "GOOG", "META", "MRNA", "NVDA", "PLTR", "TSLA", "WEWKQ", "ZI"]

collator = DataCollator(tickers=tickers, start_date=start_date, end_date=end_date, news_db_url=NEWS_DATA_DB)

In [16]:
tickers_df = collator.get_tickers_data()

100%|██████████| 448/448 [00:00<00:00, 93952.41it/s]


In [21]:
tickers_df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Headlines
0,2022-11-28,144.333235,145.824897,142.583023,143.418350,69246000,0.0,0.0,AAPL,NaN
1,2022-11-29,143.487983,144.005097,139.569896,140.385330,83763800,0.0,0.0,AAPL,"Stock Market Rally Awaits Fed Chief Powell, Ke..."
2,2022-11-30,140.614017,147.893336,139.768751,147.207169,111380900,0.0,0.0,AAPL,Elon Musk says Apple never considered removing...
3,2022-12-01,147.386191,148.301076,145.795079,147.485626,71250400,0.0,0.0,AAPL,Coinbase says Apple forced it to remove NFT wa...
4,2022-12-02,145.148696,147.177350,144.840406,146.988403,65447400,0.0,0.0,AAPL,Weekly Roundup | Apple (AAPL) Dips More Than B...


In [19]:
tickers_df.isnull().sum()

Date              0
Open              0
High              0
Low               0
Close             0
Volume            0
Dividends         0
Stock Splits      0
Ticker            0
Headlines       464
dtype: int64

In [20]:
tickers_df.shape

(3159, 10)

In [40]:
null_values_df = tickers_df[tickers_df.isnull().any(axis=1)]

In [41]:
null_values_df.Ticker.value_counts()

WEWKQ    218
ZI       117
DUOL      99
PLTR       9
ABNB       4
MRNA       4
COIN       3
AMZN       2
GOOG       2
META       2
NVDA       2
AAPL       1
TSLA       1
Name: Ticker, dtype: int64

In [47]:
null_value_tickers = list(null_values_df.Ticker.value_counts().index)
null_value_tickers

['WEWKQ',
 'ZI',
 'DUOL',
 'PLTR',
 'ABNB',
 'MRNA',
 'COIN',
 'AMZN',
 'GOOG',
 'META',
 'NVDA',
 'AAPL',
 'TSLA']

In [49]:
for ticker in null_value_tickers:
    n_data_points = tickers_df[tickers_df.Ticker == ticker].shape[0]
    null_data_points = null_values_df[null_values_df.Ticker == ticker].shape[0]

    percent_null_values = (null_data_points / n_data_points) * 100
    print(f"Percentage of null values for {ticker} is {percent_null_values:.2f}%")

Percentage of null values for WEWKQ is 89.71%
Percentage of null values for ZI is 48.15%
Percentage of null values for DUOL is 40.74%
Percentage of null values for PLTR is 3.70%
Percentage of null values for ABNB is 1.65%
Percentage of null values for MRNA is 1.65%
Percentage of null values for COIN is 1.23%
Percentage of null values for AMZN is 0.82%
Percentage of null values for GOOG is 0.82%
Percentage of null values for META is 0.82%
Percentage of null values for NVDA is 0.82%
Percentage of null values for AAPL is 0.41%
Percentage of null values for TSLA is 0.41%


In [50]:
tickers_df.dropna(inplace=True)

In [54]:
# As the data for WeWork, Zoom and Duolingo has a lot of null values, I'm dropping those companies data.

tickers_df = tickers_df[~tickers_df.Ticker.isin(["WEWKQ", "ZI", "DUOL"])]

In [55]:
tickers_df.shape

(2695, 10)

In [63]:
tickers_df.Ticker.nunique()

10

In [66]:
# Saving the collected dataset

FAI_PROJECT_DATA = os.path.join(DATA_DIR, "fai_project_data.csv")

tickers_df.to_csv(FAI_PROJECT_DATA, index=False)

In [68]:
FAI_PROJECT_DATA

'data/fai_project_data.csv'